[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ds-reboot/python-first-part/blob/main/notebooks/hometasks/Task1_dmrf.ipynb)

# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [138]:
import requests
from pprint import pprint
from time import sleep
from random import random
import pandas as pd
from tqdm import tqdm
import sqlite3
from pathlib import Path

## Список объектов

In [13]:
# Defining parameters for url requests
params = {
    'offset': 0,
    'limit': 500,
    'sortField': 'devId.devShortCleanNm',
    'sortType': 'asc',
    'objStatus': 0
}
url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object'

In [6]:
objids = []
while True:
    with requests.get(url, params) as res:
        print(params['offset'])
        if res.status_code != 200:
            break
        houses = res.json()['data']['list']
        if len(houses) == 0:
            break
        objids.extend([x['objId'] for x in houses])
        params['offset'] += params['limit']
        sleep(random())

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000


# Данные по определенному объекту

In [8]:
len(objids)

10581

In [10]:
objids[:10]

[29679, 13383, 13397, 13399, 13398, 13391, 36912, 44076, 23494, 27476]

In [19]:
house_data = []
for obj in tqdm(objids):
    with requests.get(url.replace('kn/', '') + '/' + str(obj)) as res:
        if res.status_code != 200:
            print(res.status_code)
            break
        house_data.append(res.json()['data'])

100%|████████████████████████████████████████████████████████████████████████████| 10581/10581 [24:58<00:00,  7.06it/s]


In [20]:
df = pd.json_normalize(house_data)

In [23]:
df.shape

(10581, 108)

In [51]:
df.columns

Index(['id', 'pdId', 'region', 'address', 'objElemLivingCnt',
       'objReady100PercDt', 'wallMaterialShortDesc', 'objLkFinishTypeDesc',
       'objLkFreePlanDesc', 'objElemParkingCnt',
       ...
       'developer.bankruptStage.bankruptStageDesc',
       'developer.bankruptStage.bankruptLawUrl',
       'developer.orgBankruptMsgDttm', 'metro.id', 'metro.name', 'metro.line',
       'metro.color', 'metro.time', 'metro.isWalk', 'metro.colors'],
      dtype='object', length=108)

In [31]:
df.to_excel('house_data.xlsx', index=False)

In [32]:
df.to_pickle('house_data.pkl')

In [29]:
df.to_csv('house_data.csv', index=False)

In [112]:
with sqlite3.connect('house_data.db') as conn:
    columns = df.columns[:20].append(df.columns[21:33]).append(df.columns[34:107]).append(df.columns[108:])
    df[columns].to_sql(name='houses', con=conn, index=False, if_exists='replace')

In [154]:
# 50 GBs of happiness)))
print(df.columns[20])
with requests.Session() as s:
    for value in tqdm(df.iloc[:, 20]):
        if len(value) > 0:
            for i, pict in enumerate(value):
                path = 'imgs/' + str(pict['objId']) + '_' + str(i) + '.jpg'
                if not Path(path).is_file():
                    if 'objRenderPhotoUrl' in pict:
                        img_data = s.get(pict['objRenderPhotoUrl']).content
                        with open(path, 'wb') as handler:
                            handler.write(img_data)

photoRenderDTO


100%|██████████████████████████████████████████████████████████████████████████| 10581/10581 [00:01<00:00, 8336.62it/s]


In [126]:
print(df.columns[33])
for value in df.iloc[:, 33]:
    if len(value) > 0:
        print(value)

objectTransportInfo
[{'metroStationNm': 'Золотая нива', 'isMetroStationDistWalking': False, 'metroStationDistTime': 38.25, 'metroStationDistLength': 2.166, 'metroLineColorHexCode': '0A6F20', 'metroLineColorsHexCode': ['#0A6F20']}]


In [123]:
print(df.columns[107])
for value in df.iloc[:, 107]:
    if isinstance(value, list):
        print(value)

metro.colors
['#0A6F20']
